In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


/home/pashaprokaz/.cache/pypoetry/virtualenvs/mts-test-case-m_pjxXM2-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [1]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2-7B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

NameError: name 'FastLanguageModel' is not defined

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "pashaprokaz/qwen-7b-instruct-hotel-booking-4bit-v2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth: Fast Qwen2 patching release 2024.6
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 12.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.13it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.6 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files = "train_data.json", split = "train")


Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


In [5]:
def transform_example(example):
    conversations = [
        {'from': 'human', 'value': example['instruction']},
        {'from': 'gpt', 'value': example['output']}
    ]
    return {'conversations': conversations}

# Применение функции к каждому элементу датасета
transformed_dataset = dataset.map(transform_example, remove_columns=['instruction', 'input', 'output'])
transformed_dataset = transformed_dataset.map(formatting_prompts_func, batched = True,)

In [6]:
transformed_dataset[5]["conversations"]

[{'from': 'human',
  'value': 'I need a hotel with a pool in Dubai for my family vacation from September 20 to 25; please include activities for kids.'},
 {'from': 'gpt',
  'value': "{'name': 'search_hotels_tool', 'arguments': {'location': 'Dubai', 'checkin_date': '2024-09-20', 'checkout_date': '2024-09-25', 'adults_number': 2, 'children_number': 2, 'min_rating': 8, 'order_by': 'popularity'}}"}]

In [7]:
transformed_dataset[5]["text"]

"<|im_start|>user\nI need a hotel with a pool in Dubai for my family vacation from September 20 to 25; please include activities for kids.<|im_end|>\n<|im_start|>assistant\n{'name': 'search_hotels_tool', 'arguments': {'location': 'Dubai', 'checkin_date': '2024-09-20', 'checkout_date': '2024-09-25', 'adults_number': 2, 'children_number': 2, 'min_rating': 8, 'order_by': 'popularity'}}<|im_end|>\n"

In [9]:
transformed_dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 516
})

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = transformed_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 180,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 516 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 180
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,1.654900
2,1.732400
3,1.709400
4,1.618800
5,1.433200
6,1.301400
7,1.145900
8,0.962300
9,0.749700
10,0.628300


In [10]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference


In [13]:
import json

with open("val_data.json", "r") as f:
    val_data = json.load(f)

len(val_data)

130

In [14]:
import re

def extract_text_between_tags(text, start_tag, end_tag):
    start_tag = re.escape(start_tag)
    end_tag = re.escape(end_tag)
    
    pattern = f"{start_tag}(.*?){end_tag}"
    
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return matches[-1]
    else:
        return None



def generate_answer(test_str):
    messages = [
        {"from": "human", "value": test_str},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True)
    
    result = tokenizer.batch_decode(outputs)[0]
    return extract_text_between_tags(result, start_tag = '<|im_start|>assistant\n', end_tag="<|im_end|>")


In [15]:
prompt = """
Current time: 2024-06-18 17:45:12 UTC (Tuesday)
You are an assistant who can search and book a hotel for a user.
Hotel search and reservations are made through the use of the following tools:

search_hotels_tool(location: str, checkin_date: str, checkout_date: str, adults_number: int, children_number: int = 0, order_by: str = 'popularity', min_rating=None, min_price=None, max_price=None) -> List - Search for hotels in a given parameters.
book_hotel_tool(id: int) -> str - Book hotel by a provided id. Id can be obtained from search_hotels_mock.

 Here is a list of possible parameters and their values:
-location: str -> often just the name of the city
-checkin_date: str -> format: YYYY-MM-DD
-checkout_date: str -> format: YYYY-MM-DD
-adults_number: int
-children_number: int
-min_rating: int -> must be in the range from 0 to 10
-min_price: int
-max_price: int
-order_by: str -> possible values: popularity, price, rating
-id: int -> it is used exclusively when booking, when the search has already been called

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding to the argument names and the values corresponding to the requested values.
If the user has not provided some important information, you still need to send a json blob.
Don't make up the argument values yourself! Take only what the user specified!

{user_input}
"""


In [16]:
prompt.format(user_input=val_data[0]['instruction'])

"\nCurrent time: 2024-06-18 17:45:12 UTC (Tuesday)\nYou are an assistant who can search and book a hotel for a user.\nHotel search and reservations are made through the use of the following tools:\n\nsearch_hotels_tool(location: str, checkin_date: str, checkout_date: str, adults_number: int, children_number: int = 0, order_by: str = 'popularity', min_rating=None, min_price=None, max_price=None) -> List - Search for hotels in a given parameters.\nbook_hotel_tool(id: int) -> str - Book hotel by a provided id. Id can be obtained from search_hotels_mock.\n\n Here is a list of possible parameters and their values:\n-location: str -> often just the name of the city\n-checkin_date: str -> format: YYYY-MM-DD\n-checkout_date: str -> format: YYYY-MM-DD\n-adults_number: int\n-children_number: int\n-min_rating: int -> must be in the range from 0 to 10\n-min_price: int\n-max_price: int\n-order_by: str -> possible values: popularity, price, rating\n-id: int -> it is used exclusively when booking, wh

In [9]:
val_data[0]

{'instruction': 'I want to book a hotel in Seoul with a view of the city, check-in on May 1st, check-out on May 6th, for a couple.',
 'input': '',
 'output': "{'name': 'search_hotels_tool', 'arguments': {'location': 'Seoul', 'checkin_date': '2024-05-01', 'checkout_date': '2024-05-06', 'adults_number': 2, 'order_by': 'popularity'}}"}

In [17]:
generate_answer(prompt.format(user_input=val_data[0]['instruction']))

'{"name": "search_hotels_tool", "arguments": {"location": "Seoul", "checkin_date": "2024-05-01", "checkout_date": "2024-05-06", "adults_number": 2, "order_by": "popularity"}}'

In [18]:
from tqdm import tqdm

results1 = []
for item in tqdm(val_data):
    results1.append(generate_answer(prompt.format(user_input=item['instruction'])))

100%|██████████| 130/130 [07:37<00:00,  3.52s/it]


In [19]:
with open('val_results_qwen7b-hotel-instruct-v2.json', 'w') as f:
    json.dump(results1, f, indent =4)

In [13]:
with open('val_results_qwen7b-hotel-instruct.json', 'w') as f:
    json.dump(results1, f, indent =4)

In [31]:
with open('val_results_qwen7b.json', 'w') as f:
    json.dump(results1, f, indent=4)

In [15]:
results1[0]

In [20]:
import re


def transform_empty_strings_to_none(d):
    return {k: (v if v != "" else None) for k, v in d.items()}


# Define a function to find the last complete JSON object
def find_last_json(text):
    stack = []
    last_index = -1

    # Iterate over the text to find the position of the last complete JSON object
    for i, char in enumerate(text):
        if char == "{":
            stack.append(i)
        elif char == "}":
            if stack:
                start = stack.pop()
                if not stack:
                    last_index = i

    if last_index != -1:
        json_str = text[start : last_index + 1]
        return json_str
    return None


def remove_trailing_commas(json_like: str):
    json_like = re.sub(",\s*}", "}", json_like)
    json_like = re.sub(",\s*\]", "]", json_like)
    return json_like


def remove_comments(text):
    text = re.sub(r"//.*?\n|/\*.*?\*/", "", text, flags=re.S)
    return text


def dict_to_json(d):
    try:
        result = d.replace("None", "null")
        result = result.replace("'", '"')
        result = remove_trailing_commas(result)
        result = find_last_json(result)
        result = remove_comments(result)
        result = json.loads(result, object_hook=transform_empty_strings_to_none)
        return result
    except Exception as e:
        return None

In [21]:
results1_json = []
for item in results1:
    results1_json.append(dict_to_json(item))


In [34]:
with open('val_results_qwen7b.json', 'w') as f:
    json.dump(results1_json, f, indent=4)

In [16]:
with open('val_results_qwen7b-hotel-instruct.json', 'w') as f:
    json.dump(results1_json, f, indent=4)

In [22]:
with open('val_results_qwen7b-hotel-instruct-v2.json', 'w') as f:
    json.dump(results1_json, f, indent =4)

In [17]:
sum(item is None for item in results1_json)

4

In [24]:
sum(item is None for item in results1_json)

1

In [25]:
if True: model.push_to_hub_merged("pashaprokaz/qwen-7b-instruct-hotel-booking-4bit-v2", tokenizer, save_method = "merged_4bit_forced")

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...


/home/pashaprokaz/.cache/pypoetry/virtualenvs/mts-test-case-m_pjxXM2-py3.10/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]


































































































































































































































































































































































































































































































































































































































































































































model-00002-of-00002.safetensors: 100%|██████████| 1.09G/1.09G [03:13<00:00, 5.63MB/s]























































































































































Saved merged_4bit model to https://huggingface.co/pashaprokaz/qwen-7b-instruct-hotel-booking-4bit-v2
